In [1]:
!pip install jdatetime

In [2]:
import jdatetime

# Example Shamsi date
shamsi_date = jdatetime.date(1402, 5, 7)  # Shamsi date: 1402-05-07

# Convert Shamsi date to Miladi date
miladi_date = shamsi_date.togregorian()

# Print the Miladi date
print(f"Miladi date: {miladi_date.year}-{miladi_date.month}-{miladi_date.day}")


Miladi date: 2023-7-29


In [3]:
import finpy_tse as fpy
import os
import pandas as pd
import numpy as np
import pickle

In [13]:
# Get the name of the text file from user input
# file_name = input("Enter the name of the text file: ")  
file_name = './main_stocks.txt'  
# start_date = input("Enter start_date (e.g 2011-01-01)")
start_date = '1403-01-01'
# end_date = input("Enter end_date (e.g 2021-01-01)")
end_date = '1403-04-01'

path_to_save_raw = './data/raw_data/stocks/'
path_to_save_processed = './data/processed_data/stocks/'

def read_stockList(file_name):
    print(file_name)
    global stocks_list
    try:
        with open(file_name, 'r',encoding='utf-8') as file:
            names_str = file.read()
        stocks_list = (names_str.split(','))
        return stocks_list
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")

def process_stock_data(data):
    data['Final_yes'] = data['Final'].shift(1)
    data['Volume_yes'] = data['Volume'].shift(1)
    data = data.drop(columns=['No','Market','Value','Name'])
    data['high_rate'] = (data['High'] - data['Final_yes']) / data['Final_yes']
    data['low_rate'] = (data['Low'] - data['Final_yes']) / data['Final_yes']
    data["open_rate"] = (data["Open"] - data["Final_yes"]) / data["Final_yes"]
    data['earning_rate'] = (data['Close'] - data['Final_yes']) / data['Final_yes']
    data['Volume'] = (data['Volume'] - data['Volume_yes']) / data['Volume_yes']
    data['label'] = data["earning_rate"].shift(-1)
    data['Signal'] = np.where(data['earning_rate'].shift(-1) > 0, 1, 0)

    data = data.dropna()
    return data

def save_data_toCsv(stocks_list,start_date,end_date):
    available_stocks = []
    for stock in stocks_list:
        print(stock)
        data = pd.DataFrame()

        data = fpy.Get_Price_History(
            stock= stock,
            start_date=start_date,
            end_date=end_date,
            ignore_date=False,
            adjust_price=False,
            show_weekday=False,
            double_date=False)
        
        print(data.shape)
        if(data.shape[0]>50):
            available_stocks.append(stock)
            print('available_stocks', available_stocks)
            data[-50:].to_csv(path_to_save_raw+stock+'.csv', index=True)
            data = process_stock_data(data)
            data[-50:].to_csv(path_to_save_processed+stock+'.csv', index=True)
            data = data.reset_index(drop=False)
            # print(data)
            # # Combine the folder path and file name to get the full file path
            # # Check if the file exists
            df_c = data[-50:].reset_index(drop=True)
            df_c = df_c.reset_index(drop=False)
            print(df_c)
            if os.path.exists(path_to_save_raw +'historical_data.csv'):
                df_existing = pd.read_csv(path_to_save_raw +'historical_data.csv')
                print(df_c)
                df_combined = pd.concat([df_existing, df_c], ignore_index=False)
                df_combined.to_csv(path_to_save_raw +'historical_data.csv', index=False)
            else:
                df_c.to_csv(path_to_save_raw +'historical_data.csv', index=False)
    df = pd.DataFrame({'column_name': available_stocks})
    df.to_csv('output.txt', index=False, header=False, sep='\t')

stocks_list = read_stockList(file_name)
save_data_toCsv(stocks_list, start_date,end_date)

./main_stocks.txt
خودرو


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))